In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [2]:
path = 'https://raw.githubusercontent.com/FabioHodo/EIB2_ClimateRisk/main/cleaned%20data/data_imputed/imputed_knn/'
df = pd.read_csv(path +'Emissions_GHG_(fromCAIT).csv',index_col='Unnamed: 0' )
df

,Country,Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Albania,ALB,11.40,9.04,6.99,6.93,7.77,7.53,7.25,6.54,...,8.54,8.60,8.88,9.33,8.94,9.05,9.54,9.48,9.33,9.90
1,Algeria,DZA,96.00,94.87,94.50,102.03,99.78,103.99,105.30,109.39,...,156.40,166.21,167.32,179.68,189.89,193.59,203.79,211.66,209.41,211.50
2,Angola,AGO,150.61,151.28,151.61,152.39,152.54,156.90,161.39,156.01,...,173.99,180.66,186.83,220.92,185.95,203.34,185.10,207.66,182.19,176.85
3,Antigua and Barbuda,ATG,0.34,0.35,0.43,0.41,0.42,0.44,0.47,0.50,...,1.06,2.02,1.13,1.07,1.29,1.06,1.08,1.10,1.14,1.17
4,Argentina,ARG,323.84,328.83,333.22,335.73,340.81,341.39,344.46,346.24,...,468.33,452.69,440.34,450.10,464.88,462.89,456.74,460.21,469.73,469.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,VUT,0.47,0.48,0.50,0.53,0.54,0.54,0.57,0.57,...,0.70,0.70,0.71,0.75,0.75,0.75,0.81,0.81,0.84,0.84
163,Venezuela,VEN,303.14,304.92,308.93,315.26,318.02,326.54,331.00,337.59,...,265.49,261.47,291.82,323.30,343.49,373.61,359.46,352.14,352.08,338.14
164,Vietnam,VNM,25.63,28.49,31.00,37.69,43.31,49.46,56.03,63.80,...,209.45,231.71,251.16,235.76,237.24,249.25,263.12,306.26,320.67,319.54
165,Zambia,ZMB,485.43,484.96,484.28,483.00,481.44,478.95,444.46,405.54,...,471.05,412.65,460.49,432.46,455.21,468.32,440.83,473.30,495.17,495.99


In [3]:
def makeMonthly(country, scale = 20):
    '''
    Turn the yearly data into monthly using Brownian bridge algorithm
    Parameter:
        country: a string for the country name
        scale: an integer for the data scaling
    Return:
        the scaled monthly data
    '''
    
    AnnualVal = country*scale
    n = (len(AnnualVal)-1)*12
    monthly = np.zeros(n+1)
    monthlyVal = np.zeros(n)
    for i in range(1, len(AnnualVal)+1):
        monthly[(i-1)*12] = AnnualVal[i-1]

    for i in range(1, len(AnnualVal)): #1-4
        for j in range ((i-1)*12+1, (i)*12): #0 to  47
            monthly[j] = monthly[j-1] + (monthly[(i)*12] - monthly[j-1])*(1/((i*12)-(j-1)))
            + (((i*12-j)*(1)/(i*12-(j-1)))**(1/2))*np.random.randn()
            monthlyVal[j-1] = monthly[j] -monthly[j-1]
    return monthly/scale

In [4]:
def create_dataset(dataset,timestep = 6):
    data_x,data_y = [],[]
    for i in range(len(dataset)-timestep-1):
        a = dataset[i:i+timestep,0]
        data_x.append(a)
        data_y.append(dataset[i+timestep,0])
    return np.array(data_x),np.array(data_y)

In [5]:
def predict(num_prediction, model, test_data, look_back):
    prediction_list = test_data[-look_back:]
    
    for i in range(num_prediction):
        x = prediction_list[-look_back:]
        #print(x)
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back:]
        
    return prediction_list

In [6]:
def FuturePrediction(country,look_back,num_prediction):
    '''
    Make prediction for the data of one country to 2030
    Parameter:
        country: a string that specifies the country name
    Return:
        prediction: a list of values for future prediction of the data for each year
    '''
    data = df[df['Country']==country]
    data = data.loc[:,"1990":"2017"].T
    monthly = makeMonthly(np.array(data), 10)
    monthly = np.reshape(monthly, (-1, 1))
    
    # Feature Scaling
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(monthly)
    
    #split train and test dataset
    train_size = int(len(data)*0.7)
    test_size = len(data)-train_size
    train_data, test_data = data[0:train_size,:],data[train_size:len(data),:]
    
    x_train, y_train = create_dataset(train_data,timestep = look_back)
    x_test, y_test = create_dataset(test_data,timestep = look_back)
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
    
    # Create the Stacked LSTM model
    model = Sequential()

    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    # Fitting the RNN to the Training set
    model.fit(x_train, y_train, validation_data = (x_test, y_test) ,epochs = 50, batch_size = 10,verbose =1,
         callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
    
    # Make monthly predictions into the future
    x = predict(num_prediction, model, test_data, look_back)
    prediction = scaler.inverse_transform(x.reshape(-1,1))
    # Make yearly predictions into the future
    annual_prediction = prediction[11:num_prediction:12]
    return annual_prediction

In [7]:
def PredictAll(df,look_back,num_prediction):
    '''
    Make predictions to 2030 for all the countries with valid data
    Parameter:
        df: a dataframe with the data
    Return:
        df: a dataframe with the original data and the predictions
    '''
    countries = df['Country']
    world_prediction = []
    for country in countries:
        #print(FuturePrediction(country,look_back,num_prediction))
        world_prediction.append(FuturePrediction(country,look_back,num_prediction))
    world_prediction = np.squeeze(np.array(world_prediction), axis = (2,))
    years = ['2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
    #print(world_prediction)
    #print(world_prediction.shape)
    #print(world_prediction[:, 1])
    for i in range(len(years)):
        df[years[i]] = world_prediction[:, i]
    return df


In [8]:
ghg_prediction = PredictAll(df,look_back=5,num_prediction=13*12)
ghg_prediction

Epoch 1/50
23/23 [==============================] - 3s 60ms/step - loss: 0.1160 - val_loss: 0.0028
Epoch 2/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0371 - val_loss: 0.0192
Epoch 3/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0291 - val_loss: 0.0113
Epoch 4/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0224 - val_loss: 0.0088
Epoch 5/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0188 - val_loss: 0.0022
Epoch 6/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0145 - val_loss: 3.9045e-04
Epoch 7/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0107 - val_loss: 6.0627e-04
Epoch 8/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0088 - val_loss: 7.6376e-04
Epoch 9/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0110 - val_loss: 0.0100
Epoch 10/50
23/23 [==============================] - 1s 25ms/step - loss: 0.0121 - val_loss: 0.00

Epoch 16/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0018 - val_loss: 0.0103
Epoch 17/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0017 - val_loss: 0.0137
Epoch 18/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0015 - val_loss: 0.0109
Epoch 19/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0022 - val_loss: 0.0204
Epoch 20/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0018 - val_loss: 0.0291
Epoch 21/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0019 - val_loss: 0.0100
Epoch 22/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0021 - val_loss: 0.0054
Epoch 23/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0015 - val_loss: 0.0247
Epoch 24/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0059 - val_loss: 0.0165
Epoch 25/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0015 - val_loss: 0.0090


23/23 [==============================] - 1s 24ms/step - loss: 0.0154 - val_loss: 0.0043
Epoch 7/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0118 - val_loss: 0.0019
Epoch 8/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0147 - val_loss: 0.0033
Epoch 9/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0156 - val_loss: 0.0022
Epoch 10/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0138 - val_loss: 0.0027
Epoch 11/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0136 - val_loss: 0.0018
Epoch 12/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0156 - val_loss: 0.0046
Epoch 13/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0131 - val_loss: 0.0023
Epoch 14/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0122 - val_loss: 0.0019
Epoch 15/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0166 - val_loss: 0.0087
Epoch 16/50
23/

Epoch 35/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0035 - val_loss: 0.0149
Epoch 36/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0044 - val_loss: 0.0141
Epoch 1/50
23/23 [==============================] - 3s 64ms/step - loss: 0.0286 - val_loss: 0.2441
Epoch 2/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0142 - val_loss: 0.2419
Epoch 3/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0122 - val_loss: 0.1982
Epoch 4/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0084 - val_loss: 0.0892
Epoch 5/50
23/23 [==============================] - 1s 26ms/step - loss: 0.0034 - val_loss: 0.0060
Epoch 6/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 7/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0021 - val_loss: 0.0087
Epoch 8/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0020 - val_loss: 3.1869e-04
Epoc

23/23 [==============================] - 1s 24ms/step - loss: 0.0123 - val_loss: 0.0233
Epoch 9/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0111 - val_loss: 0.0070
Epoch 10/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0160 - val_loss: 0.0113
Epoch 11/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0113 - val_loss: 0.0106
Epoch 12/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0102 - val_loss: 0.0070
Epoch 13/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0108 - val_loss: 0.0071
Epoch 14/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0113 - val_loss: 0.0064
Epoch 15/50
23/23 [==============================] - 1s 23ms/step - loss: 0.0134 - val_loss: 0.0077
Epoch 16/50
23/23 [==============================] - 1s 25ms/step - loss: 0.0092 - val_loss: 0.0088
Epoch 17/50
23/23 [==============================] - 1s 25ms/step - loss: 0.0119 - val_loss: 0.0079
Epoch 18/50
2

...

23/23 [==============================] - 1s 25ms/step - loss: 0.0026 - val_loss: 0.0101
Epoch 9/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0042 - val_loss: 6.4164e-04
Epoch 10/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0031 - val_loss: 8.6222e-04
Epoch 11/50
23/23 [==============================] - 1s 26ms/step - loss: 0.0026 - val_loss: 4.3886e-04
Epoch 12/50
23/23 [==============================] - 1s 25ms/step - loss: 0.0018 - val_loss: 0.0012
Epoch 13/50
23/23 [==============================] - 1s 25ms/step - loss: 0.0020 - val_loss: 0.0014
Epoch 14/50
23/23 [==============================] - 1s 26ms/step - loss: 0.0031 - val_loss: 0.0013
Epoch 15/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0030 - val_loss: 2.5731e-04
Epoch 16/50
23/23 [==============================] - 1s 26ms/step - loss: 0.0034 - val_loss: 1.1153e-04
Epoch 17/50
23/23 [==============================] - 1s 24ms/step - loss: 0.0028 - val_loss: 

Epoch 3/50
23/23 [==============================] - 1s 34ms/step - loss: 0.0365 - val_loss: 0.1124
Epoch 4/50
23/23 [==============================] - 1s 32ms/step - loss: 0.0286 - val_loss: 0.0795
Epoch 5/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0193 - val_loss: 0.0554
Epoch 6/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0132 - val_loss: 0.0115
Epoch 7/50
23/23 [==============================] - 1s 31ms/step - loss: 0.0081 - val_loss: 0.0058
Epoch 8/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0066 - val_loss: 0.0045
Epoch 9/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0061 - val_loss: 0.0028
Epoch 10/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0053 - val_loss: 0.0029
Epoch 11/50
23/23 [==============================] - 1s 30ms/step - loss: 0.0064 - val_loss: 0.0044
Epoch 12/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0085 - val_loss: 0.0225
Epoch 1

Epoch 11/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0153 - val_loss: 0.0024
Epoch 12/50
23/23 [==============================] - 1s 31ms/step - loss: 0.0157 - val_loss: 0.0044
Epoch 13/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0186 - val_loss: 0.0180
Epoch 14/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0197 - val_loss: 0.0113
Epoch 15/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0167 - val_loss: 0.0015
Epoch 16/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0144 - val_loss: 0.0063
Epoch 17/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0143 - val_loss: 0.0102
Epoch 18/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0115 - val_loss: 0.0033
Epoch 19/50
23/23 [==============================] - 1s 31ms/step - loss: 0.0149 - val_loss: 0.0046
Epoch 20/50
23/23 [==============================] - 1s 28ms/step - loss: 0.0132 - val_loss: 0.0035


23/23 [==============================] - 1s 27ms/step - loss: 0.0348 - val_loss: 0.0094
Epoch 19/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0237 - val_loss: 0.0077
Epoch 20/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0249 - val_loss: 0.0043
Epoch 21/50
23/23 [==============================] - 1s 29ms/step - loss: 0.0272 - val_loss: 0.0237
Epoch 22/50
23/23 [==============================] - 1s 27ms/step - loss: 0.0229 - val_loss: 0.0095


,Country,Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
0,Albania,ALB,11.40,9.04,6.99,6.93,7.77,7.53,7.25,6.54,...,10.689220,10.794928,10.869599,10.920926,10.955534,10.978563,10.993749,11.003706,11.010208,11.014444
1,Algeria,DZA,96.00,94.87,94.50,102.03,99.78,103.99,105.30,109.39,...,158.522908,155.852320,153.939516,152.530283,151.471621,150.665126,150.044319,149.562812,149.187146,148.892755
2,Angola,AGO,150.61,151.28,151.61,152.39,152.54,156.90,161.39,156.01,...,184.127724,184.127724,184.127724,184.127724,184.127724,184.127724,184.127724,184.127724,184.127724,184.127724
3,Antigua and Barbuda,ATG,0.34,0.35,0.43,0.41,0.42,0.44,0.47,0.50,...,5.792424,5.804359,5.805221,5.805283,5.805287,5.805287,5.805287,5.805287,5.805287,5.805287
4,Argentina,ARG,323.84,328.83,333.22,335.73,340.81,341.39,344.46,346.24,...,455.567713,455.364268,455.278267,455.241893,455.226449,455.219901,455.217136,455.215962,455.215475,455.215241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,VUT,0.47,0.48,0.50,0.53,0.54,0.54,0.57,0.57,...,0.566036,0.544625,0.527044,0.513043,0.502353,0.494534,0.489021,0.485245,0.482713,0.481041
163,Venezuela,VEN,303.14,304.92,308.93,315.26,318.02,326.54,331.00,337.59,...,287.271550,282.941234,280.202473,278.506108,277.469676,276.841963,276.463821,276.236757,276.100690,276.019241
164,Vietnam,VNM,25.63,28.49,31.00,37.69,43.31,49.46,56.03,63.80,...,199.155761,196.525829,195.040765,194.187381,193.692183,193.403091,193.233846,193.134486,193.076172,193.041845
165,Zambia,ZMB,485.43,484.96,484.28,483.00,481.44,478.95,444.46,405.54,...,458.216322,456.559369,455.545866,454.919151,454.529059,454.285243,454.132475,454.036618,453.976427,453.938567


In [9]:
# Save the dataset as csv for model fitting later
ghg_prediction.to_csv("GHG emission to 2030 (LSTM).csv")